In [22]:
import pandas as pd
import numpy as np
import ast

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

from sklearn import svm
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

from sklearn_hierarchical_classification.classifier import HierarchicalClassifier
from sklearn_hierarchical_classification.constants import ROOT

from util_koyak import col_selector_hiera, grid_estimator
from tqdm import tqdm_notebook

#### Importing features and selecting them.

In [23]:
df_genre = pd.read_csv("features_genre_HIERA_min_5.csv", index_col=0)
df_mfcc_cont = col_selector_hiera(['mfcc','contrast'], df_genre)

In [24]:
df_mfcc_cont.head()

,"('mfcc', 'kurtosis', '01')","('mfcc', 'kurtosis', '02')","('mfcc', 'kurtosis', '03')","('mfcc', 'kurtosis', '04')","('mfcc', 'kurtosis', '05')","('mfcc', 'kurtosis', '06')","('mfcc', 'kurtosis', '07')","('mfcc', 'kurtosis', '08')","('mfcc', 'kurtosis', '09')","('mfcc', 'kurtosis', '10')",...,"('spectral_contrast', 'skew', '06')","('spectral_contrast', 'skew', '07')","('spectral_contrast', 'std', '01')","('spectral_contrast', 'std', '02')","('spectral_contrast', 'std', '03')","('spectral_contrast', 'std', '04')","('spectral_contrast', 'std', '05')","('spectral_contrast', 'std', '06')","('spectral_contrast', 'std', '07')",genre_selected
track_id,,,,,,,,,,,,,,,,,,,,,
2,3.856789,1.541901,0.000816,0.330728,0.118731,-0.342687,-0.259252,0.146735,0.410656,-0.162872,...,1.690805,-1.571900,4.541557,4.321756,3.936234,3.144537,3.310087,3.096597,7.622641,21
3,4.296755,1.399977,0.112535,-0.211170,0.032953,-0.023489,0.150404,0.046454,0.033484,-0.064596,...,0.391984,-1.461130,4.430571,4.517010,4.631938,3.747272,4.085845,3.351878,7.611158,21
5,2.624517,2.415293,0.440233,-0.782131,-0.771069,-0.724216,0.090260,0.152119,0.261731,-0.608905,...,0.366885,-1.271206,4.942019,4.376371,4.262785,3.190465,3.102941,3.087401,8.494939,21
10,5.076893,1.161854,2.095651,1.372743,-0.203574,-0.345354,-0.529139,0.561974,0.281350,-0.150672,...,1.257145,-0.592026,5.561779,4.008956,3.690036,2.646124,3.457347,4.693093,8.396724,10
134,1.559808,0.840775,-0.171289,-0.265671,-0.279361,0.190866,-0.192000,0.422766,0.044087,-0.003510,...,0.683411,-0.736473,4.736264,4.715623,4.440975,3.465022,3.858569,3.905518,8.469674,21


In [25]:
X = df_mfcc_cont.drop("genre_selected",axis=1)
y = df_mfcc_cont["genre_selected"]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.10, random_state=4444)

In [27]:
sca = StandardScaler()
X_train_sca = sca.fit_transform(X_train)
X_test_sca = sca.transform(X_test);

Creating the class hierarchy

In [28]:
df_genres = pd.read_csv("genres.csv")

In [29]:
class_hierarchy = {}
for index in range(df_genres["genre_id"].shape[0]):
    try:
        class_hierarchy[df_genres["parent"].iloc[index]].append(df_genres["genre_id"].iloc[index])
    except:    
        class_hierarchy[df_genres["parent"].iloc[index]] = [df_genres["genre_id"].iloc[index]]
                     
class_hierarchy[ROOT]=class_hierarchy[0]
del class_hierarchy[0]

### Hierarchical classification

In [31]:
base_estimator = make_pipeline(
 #   TruncatedSVD(n_components=100),
    RandomForestClassifier()
    )

In [32]:
clf = HierarchicalClassifier(
    base_estimator = base_estimator,
     class_hierarchy=class_hierarchy,
    progress_wrapper=tqdm_notebook)


In [33]:
clf.fit(X_train_sca, y_train)
y_pred = clf.predict(X_test)

HBox(children=(IntProgress(value=0, description='Building features', max=164), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Training base classifiers', max=164), HTML(value='')))

_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 86
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 3
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 651
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 109
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 19
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 181
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 468
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 16
_trai

In [34]:
accuracy_score(y_test, y_pred)

0.21383352872215708

In [35]:
y_pred_train = clf.predict(X_train)
accuracy_score(y_train, y_pred_train)

0.21363067944977074

In [36]:
base_estimator = make_pipeline(
    TruncatedSVD(n_components=100),
    RandomForestClassifier()
    )

In [37]:
clf = HierarchicalClassifier(
    base_estimator = base_estimator,
     class_hierarchy=class_hierarchy,
    progress_wrapper=tqdm_notebook)


In [38]:
clf.fit(X_train_sca, y_train)
y_pred = clf.predict(X_test)

HBox(children=(IntProgress(value=0, description='Building features', max=164), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Training base classifiers', max=164), HTML(value='')))

_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 86
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 3
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 651
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 109
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 19
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 181
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 468
_train_local_classifier() - not enough training data available to train, classification in branch will terminate at node 16
_trai

In [39]:
accuracy_score(y_test, y_pred)

0.03610785463071512

In [40]:
y_pred_train = clf.predict(X_train)
accuracy_score(y_train, y_pred_train)

0.036551688203418094